Python Test (Homework #1)

<b>#1: PyART Test:</b> Using the code below, create a plot of WSR-88D Radar data. What variable is plotted and what do you see? Try altering the ploting domain (see lat/lon variables) to determine where this storm is occuring. Download the radar file from blackboard. Try 

In [ ]:
## This section imports modules needed to run the code
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyart
import cartopy.crs as ccrs


## End import section

radar_file='data//radar//KTLX20130531_231434_V06.gz' ## Update directory as needed

image_file='output//KTLX20130531_231434_Z.png'   ## Will output to python output subdirectory

# read in the file, create a RadarMapDisplay object
my_radar = pyart.io.read(radar_file)
display = pyart.graph.RadarMapDisplay(my_radar)
# plot the first (0) tilt

#Variables you can plot:
# 'reflectivity' , vmin=-20 , vmax= depends on time of year. 75 for convection, 30-40 for winter
# 'differential_reflectivity' vmin =  -4,vmax = 8
# 'velocity' choose vmin and vmax based off conditions. -30 to 30 m/s is good starting point
# 'spectrum_width'  vmin=0,  vmax=10
# 'cross_correlation_ratio'   vmin=0,vmax=1
# 'differential_phase'  vmin=0 vmax=180
#You can change the variable to plot in the display.plot_ppi_map command. 

#Default setting center display on radar and go out 2 degrees/lat/lon from radar.  
#Once you do an overview plot, create a zoomed in plot by changing the center lat/lon and decreasing the
#increment from 2 to 0.5 or 0.25 degrees. 

center_lon=my_radar.longitude['data'][0] #Grab center coordinates of radar
center_lat=my_radar.latitude['data'][0]  #Grab center coordinates of radar

#Define range around center coordinates to plot.  This is in degrees latitude/longitude
min_lat = center_lat-1.0
max_lat = center_lat+1.0
min_lon = center_lon-1.0
max_lon = center_lon+1.0

projection = ccrs.PlateCarree()
display = pyart.graph.RadarMapDisplay(my_radar)
fig = plt.figure(figsize=(10, 10),dpi=600)  #Create figure
ax1 = fig.add_subplot(111,projection=projection) 
display.plot_ppi_map('reflectivity', 1, vmin=10, vmax=70,
                     min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                     resolution='10m', projection=projection,colorbar_flag=0,
                     fig=fig, lat_0=my_radar.latitude['data'][0],
                     lon_0=my_radar.longitude['data'][0],title = 'KTLX 2314 UTC 5/31/13')
#Setup colorbar
colorbar_panel_axes = [0.15, 0.04, 0.725, 0.05]
cbax = fig.add_axes(colorbar_panel_axes)
display.plot_colorbar(cax=cbax,orient='horizontal', label='Equivalent Reflectivity Factor (dBZ)')

display.plot_point(center_lon, center_lat)
plt.savefig(image_file,dpi=300) 

<b>#2: Example Test:</b> Using the code below, download the provided example and run the code. What does this code demonstrate?

In [ ]:
# %load http://matplotlib.org/mpl_examples/showcase/integral_demo.py
"""
Plot demonstrating the integral as the area under a curve.

Although this is a simple example, it demonstrates some important tweaks:

    * A simple line plot with custom color and line width.
    * A shaded region created using a Polygon patch.
    * A text label with mathtext rendering.
    * figtext calls to label the x- and y-axes.
    * Use of axis spines to hide the top and right spines.
    * Custom tick placement and labels.
"""
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon


def func(x):
    return (x - 3) * (x - 5) * (x - 7) + 85


a, b = 2, 9  # integral limits
x = np.linspace(0, 10)
y = func(x)

fig, ax = plt.subplots()
plt.plot(x, y, 'r', linewidth=2)
plt.ylim(ymin=0)

# Make the shaded region
ix = np.linspace(a, b)
iy = func(ix)
verts = [(a, 0)] + list(zip(ix, iy)) + [(b, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

plt.text(0.5 * (a + b), 30, r"$\int_a^b f(x)\mathrm{d}x$",
         horizontalalignment='center', fontsize=20)

plt.figtext(0.9, 0.05, '$x$')
plt.figtext(0.1, 0.9, '$y$')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_ticks_position('bottom')

ax.set_xticks((a, b))
ax.set_xticklabels(('$a$', '$b$'))
ax.set_yticks([])

plt.show()


<b>#3: Example Test: </b> Using the code below, plot a visible image from GOES-16.  What station data is plotted? What do you see?

In [ ]:
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import h5py
from satpy import Scene
from satpy.writers import get_enhanced_image
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


mpl.rcParams['axes.linewidth'] = 4 #set the value globally

#Define directoru
BASE_DIR = 'data//sat//'
date='20190551737137'
hour=date[7:11]
#Load scene, grabs all data with given start time. Foes GOES16 L1B -> C01-C16
scn = Scene(reader='abi_l1b', filenames=glob(BASE_DIR+'OR_ABI-L1b-RadC-*s'+date+'*.nc'))
chan='C02'
scn.load([chan])
var = get_enhanced_image(scn[chan]).data.squeeze()
var = var.transpose('y', 'x')

fig = plt.figure(figsize=(10,15), dpi=600)
abi_crs = scn[chan].attrs['area'].to_cartopy_crs()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-98, -94, 43.5, 50], crs=ccrs.PlateCarree())
#ax.imshow(var.data, extent=(var.x[0], var.x[-1], var.y[-1], var.y[0]), origin='upper',cmap='Greys')
ax.imshow(var.data, extent=abi_crs.bounds, origin='upper',cmap='Greys_r',transform=abi_crs)
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), edgecolor='black')
ax.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='black')
#change title accordingly
ax.set_title(hour+' UTC - 24 Feb. 2019 - Band 1 - (0.46 $\mu$m)',fontweight="bold",fontsize=12)
